In [1]:
import tensorflow as tf
import pickle
import numpy as np

/home/tianwei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
model_filepath = 'SaveModeltt/ttt.pb'

print(model_filepath)

SaveModeltt/ttt.pb


In [3]:
'''
Lode trained model.
'''
print('Loading model...')
graph = tf.Graph()
sess = tf.InteractiveSession(graph = graph)

with tf.gfile.GFile(model_filepath, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

print('Check out the input placeholders:')
nodes = [n.name + ' => ' +  n.op for n in graph_def.node if n.op in ('Placeholder')]
for node in nodes:
    print(node)

Loading model...
Check out the input placeholders:
input => Placeholder


In [4]:
# Define input tensor
model_input = tf.placeholder(tf.float32, shape=[None,40,40,3], name = 'input')


# model_input = tf.placeholder(np.float32, shape = [None, 32, 32, 3], name='input')
# dropout_rate = tf.placeholder(tf.float32, shape = [], name = 'dropout_rate')

tf.import_graph_def(graph_def, {'input': model_input})

print('Model loading complete!')

Model loading complete!


In [5]:
# Know your output node name
output_tensor = graph.get_tensor_by_name("import/logits_tensor/BiasAdd:0")
# output = sess.run(output_tensor, feed_dict =  {model_input:  X_test[0:2] })

# Loading data

In [6]:
def one_hot_encode(labels, num_class):
    n_labels = len(labels)
#     n_unique_labels = len(np.unique(labels))
    n_unique_labels = num_class
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [7]:
## GET DATA TO WORK ON
print("Start loading data")

fd = open("data_x.pkl", 'rb')
fd2 = open("data_y.pkl", 'rb')
features = pickle.load(fd)
labels = pickle.load(fd2)

print("Data loaded")

# all testing data
X_test = features[:]
Y_test = labels[:]

X_test = X_test.astype('float32')
X_test /= 255

## one hot encoding
Y_test = one_hot_encode(Y_test, 10)

print(X_test.shape)
print(Y_test.shape)

batch_size = 256

n_batches_test = Y_test.shape[0]//batch_size
print('For batch of size %d: \n %d batches in test'%(batch_size, n_batches_test))

Start loading data
Data loaded
(54154, 40, 40, 3)
(54154, 10)
For batch of size 256: 
 211 batches in test


# Testing on loaded model

In [18]:
output = sess.run(output_tensor, feed_dict =  {model_input:  X_test[0:5] })
print(output.shape)

(5, 10)


In [19]:
def predict_label(logits_output):
    predict_y = np.argmax(logits_output, axis = 1)
    return predict_y

print(predict_label(output))

[7 7 7 7 7]


In [25]:
def exp_evidence(logits): 
    return np.exp(np.clip(logits,-10,10))

def uncertainty_score( logits_output ):
    evidence = exp_evidence(logits_output)
    alpha = evidence + 1
    u_score = 10 / np.sum(alpha, axis=1, keepdims=True)  # K = num_classes = 10
    return u_score

print(uncertainty_score(output))

[[0.38743544]
 [0.377615  ]
 [0.40701267]
 [0.4156392 ]
 [0.43884096]]
